# Import Libraries

In [1]:
import pandas as pd
import numpy as np

# reading csv

In [2]:
df = pd.read_csv("C:/Users/10675048/OneDrive - LTIMindtree/CPG DOMAIN/Sample Data/NPD Data/NPD_Beverage_13_Months_Data_Regional_Hierarchy Data.csv",encoding='latin-1')
df.head(1)
#df.describe
#df.columns
#df.shape

,Date,MARKT_HIER_SHORT,MARKT_HIER_LONG,MARKT_HIER_NUM,MARKT_HIER_LEVEL_NUM,MARKT_HIER_LEVEL_NAME,PROD_HIER_SHORT,PROD_HIER_LEVEL_NUM,PROD_HIER_LEVEL_NAME,1 CCF level 0,...,Units (in 1000 ITEM),Volume (in 1000 KG),Value (in 1000),Share of Sales in Sellers in Eq. (w),Weight. Distribution (w),Price per Volume (in USD),Price per Item (in USD),Unweighted Rate of Sales Eq. (C),Numeric Distribution Selling (C),TDP.CAL
0,01-08-2021,PROVINCE 1,TOTAL WEST - OTHER WEST PROVINCE 1,1,4,FMCG PROVINCE 1 / PROVINCE 2,355 mL _ Carbonated Soft Drinks_Coco-Cool_Coco...,7,VARIANT,Beverages,...,4,0,72,13,26943,1726804,18.0,416,545,3


# Subsetting Data

In [3]:
data = df[['Date', 'MARKT_HIER_LONG','MARKT_HIER_LEVEL_NAME', 'PRODUCT TYPE', 'MANUFACTURER', 'BRAND', 'SUB-BRAND',
       'VARIANT', 'Prod_Variant_Hier', 'Units (in 1000 ITEM)',
       'Volume (in 1000 KG)', 'Value (in 1000)']]
data.head(1)

,Date,MARKT_HIER_LONG,MARKT_HIER_LEVEL_NAME,PRODUCT TYPE,MANUFACTURER,BRAND,SUB-BRAND,VARIANT,Prod_Variant_Hier,Units (in 1000 ITEM),Volume (in 1000 KG),Value (in 1000)
0,01-08-2021,TOTAL WEST - OTHER WEST PROVINCE 1,FMCG PROVINCE 1 / PROVINCE 2,Carbonated Soft Drinks,Coco-Cool,Coco-Cool,prite,Regular,Carbonated Soft Drinks_Regular,4,0,72


# Group data by Market and Product hierarchy

In [4]:
data = data.sort_values(by=['PRODUCT TYPE', 'MANUFACTURER','BRAND','SUB-BRAND','VARIANT','Prod_Variant_Hier','Date'])

data = data.groupby(['MARKT_HIER_LONG','MARKT_HIER_LEVEL_NAME','PRODUCT TYPE','MANUFACTURER','BRAND','SUB-BRAND','VARIANT','Prod_Variant_Hier','Date']).agg({'Units (in 1000 ITEM)': 'sum',
    'Volume (in 1000 KG)' : 'sum', 'Value (in 1000)': 'sum'}).reset_index()
current_time = pd.Timestamp.now().strftime('%d-%m-%Y %H_%M_%S')

data.head(1)
#data.to_csv(f"C:/Users/10675048/OneDrive - LTIMindtree/CPG DOMAIN/Sample Data/NPD Data/data_{current_time}.csv")

,MARKT_HIER_LONG,MARKT_HIER_LEVEL_NAME,PRODUCT TYPE,MANUFACTURER,BRAND,SUB-BRAND,VARIANT,Prod_Variant_Hier,Date,Units (in 1000 ITEM),Volume (in 1000 KG),Value (in 1000)
0,TOTAL WEST - OTHER WEST PROVINCE 1,FMCG PROVINCE 1 / PROVINCE 2,Alcoholic Beverages,Feineken,Amstel Hard,Amstel Hard Bier,Regular,Alcoholic Beverages _Regular,01-01-2022,1552,155,16147


# Creating unique Key for Variant combination by Market and Product

In [5]:
data['variant_combo'] = data['MARKT_HIER_LONG']+data['MARKT_HIER_LEVEL_NAME']+data['PRODUCT TYPE']+data['MANUFACTURER']+data['BRAND']+data['SUB-BRAND']+data['VARIANT'].str.strip()
data['variant_combo'].head(5)

0    TOTAL WEST - OTHER WEST PROVINCE 1FMCG PROVINC...
1    TOTAL WEST - OTHER WEST PROVINCE 1FMCG PROVINC...
2    TOTAL WEST - OTHER WEST PROVINCE 1FMCG PROVINC...
3    TOTAL WEST - OTHER WEST PROVINCE 1FMCG PROVINC...
4    TOTAL WEST - OTHER WEST PROVINCE 1FMCG PROVINC...
Name: variant_combo, dtype: object

# Data Preparation 

In [6]:
data.columns

df = data[['variant_combo','Date', 'Units (in 1000 ITEM)', 'Volume (in 1000 KG)',
       'Value (in 1000)']]
#df.head(1)

df.rename(columns = {'variant_combo':'varaintcombo','Date':'Dates','Value (in 1000)':'Sales','Volume (in 1000 KG)':'Volume','Units (in 1000 ITEM)':'Unit'}, inplace = True)
df.columns

C:\Users\10675048\AppData\Local\Temp\ipykernel_4728\4153766813.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'variant_combo':'varaintcombo','Date':'Dates','Value (in 1000)':'Sales','Volume (in 1000 KG)':'Volume','Units (in 1000 ITEM)':'Unit'}, inplace = True)


Index(['varaintcombo', 'Dates', 'Unit', 'Volume', 'Sales'], dtype='object')

# Defining objects for N-month rolling calculation

In [7]:
 rolling_data = {'Dates': [],'varaintcombo': [], 'Sales':[], 'Volume':[],'Unit':[], 'Sales_1month_Rolling': [], 'Volume_1month_Rolling': [], 'Unit_1month_Rolling': [],
                    'Sales_2month_Rolling': [], 'Volume_2month_Rolling': [], 'Unit_2month_Rolling': [],
                    'Sales_3month_Rolling': [], 'Volume_3month_Rolling': [], 'Unit_3month_Rolling': [],
                    'Sales_4month_Rolling': [], 'Volume_4month_Rolling': [], 'Unit_4month_Rolling': []}

# Defining Custom function to generate future dates

In [8]:
def get_future_date(x,d):
    date_element = d.split("-")
    future_month = int(date_element[1]) + x
    if future_month > 12:
        future_month = future_month%12
        future_year = int(date_element[2]) + 1
        return date_element[0] + "-" + str(future_month).zfill(2) + "-" + str(future_year)
    else:
        return date_element[0] + "-" + str(future_month).zfill(2) + "-" + date_element[2] 

# Defining Function to Generate the N Rolling Months - Input [dataframe, N-Months] 

In [9]:

def calculate_rolling(df,n):
    unique_variants =df['varaintcombo'].unique()
    #unique_variants
    for variant in unique_variants:
        variant_data = df[df['varaintcombo'] == variant]
        #print(variant_data['Dates'])
        unique_dates = variant_data['Dates'].unique()
        #print(unique_dates)
        for d in unique_dates:
            print(d,type(d))
            d = pd.to_datetime(d,format='%d-%m-%Y').strftime('%d-%m-%Y')
            #print(d,type(d))
            rolling_data['Dates'].append(d)
            rolling_data['varaintcombo'].append(variant)
            rolling_data['Sales'].append((variant_data[variant_data['Dates']== d]['Sales']).values[-1])
            rolling_data['Volume'].append((variant_data[variant_data['Dates']== d]['Volume']).values[-1])
            rolling_data['Unit'].append((variant_data[variant_data['Dates']== d]['Unit']).values[-1])
            roll_sales = [0]
            roll_volume = [0]
            roll_unit = [0]
            #n = n+1
            for i in range(1,n+1): 
                
                future_date = get_future_date(i,d)
                print("future:",future_date)
                if future_date in unique_dates:
                    if i ==1:
                        rolling_sales = (variant_data[variant_data['Dates']== d]['Sales']).values[-1] + (variant_data[variant_data['Dates'] == future_date]['Sales']).values[-1]
                        rolling_volume = (variant_data[variant_data['Dates']== d]['Volume']).values[-1] + (variant_data[variant_data['Dates'] == future_date]['Volume']).values[-1]
                        rolling_unit = (variant_data[variant_data['Dates']== d]['Unit']).values[-1] + (variant_data[variant_data['Dates'] == future_date]['Unit']).values[-1]
                        #print((variant_data[variant_data['Dates']== d]['Sales']).values[-1])
                        #print((variant_data[variant_data['Dates']== future_date]['Sales']).values[-1])
                        #print("rolling sum:", rolling_sales)
                        # Append rolling data to the rolling_data dictionary
                        rolling_data[f'Sales_{i}month_Rolling'].append(rolling_sales)
                        rolling_data[f'Volume_{i}month_Rolling'].append(rolling_volume)
                        rolling_data[f'Unit_{i}month_Rolling'].append(rolling_unit)
                        roll_sales[0]=rolling_sales
                        roll_volume[0] = rolling_volume
                        roll_unit[0] = rolling_unit
                        
                    else:
                        #print(roll_sales)
                        rolling_sales = roll_sales[0] + (variant_data[variant_data['Dates'] == future_date]['Sales']).values[-1]
                        rolling_volume = roll_volume[0] + (variant_data[variant_data['Dates'] == future_date]['Volume']).values[-1]
                        rolling_unit = roll_unit[0] + (variant_data[variant_data['Dates'] == future_date]['Unit']).values[-1]
                        #print((variant_data[variant_data['Dates']== d]['Sales']).values[-1])
                        #print((variant_data[variant_data['Dates']== future_date]['Sales']).values[-1])
                        #print("rolling sum:", rolling_sales)
                        # Append rolling data to the rolling_data dictionary
                        rolling_data[f'Sales_{i}month_Rolling'].append(rolling_sales)
                        rolling_data[f'Volume_{i}month_Rolling'].append(rolling_volume)
                        rolling_data[f'Unit_{i}month_Rolling'].append(rolling_unit)
                        roll_sales[0]=rolling_sales
                        roll_volume[0] = rolling_volume
                        roll_unit[0] = rolling_unit
                else:
                    if i ==1:
                        rolling_sales = (variant_data[variant_data['Dates']== d]['Sales']).values[-1] + 0
                        rolling_volume = (variant_data[variant_data['Dates']== d]['Volume']).values[-1] + 0
                        rolling_unit = (variant_data[variant_data['Dates']== d]['Unit']).values[-1] + 0
                        #print((variant_data[variant_data['Dates']== d]['Sales']).values[-1])
                        #print("GAP_rolling sum:", rolling_sales)
                        # Append rolling data to the rolling_data dictionary
                        rolling_data[f'Sales_{i}month_Rolling'].append(rolling_sales)
                        rolling_data[f'Volume_{i}month_Rolling'].append(rolling_volume)
                        rolling_data[f'Unit_{i}month_Rolling'].append(rolling_unit)
                        #print(rolling_data)
                        roll_sales[0]=rolling_sales
                        roll_volume[0] = rolling_volume
                        roll_unit[0] = rolling_unit
                    else:
                        #print(roll_sales)
                        rolling_sales = roll_sales[0] + 0
                        rolling_volume = roll_volume[0] + 0
                        rolling_unit = roll_unit[0] + 0
                        #print("GAP_rolling sum:", rolling_sales)
                        # Append rolling data to the rolling_data dictionary
                        rolling_data[f'Sales_{i}month_Rolling'].append(rolling_sales)
                        rolling_data[f'Volume_{i}month_Rolling'].append(rolling_volume)
                        rolling_data[f'Unit_{i}month_Rolling'].append(rolling_unit)
                        roll_sales[0]=rolling_sales
                        roll_volume[0]=rolling_volume
                        roll_unit[0] = rolling_unit
    return rolling_data


                        
                    

# Calling Function for calculating N-Months

In [10]:
calculate_rolling(df,4)
rd_df = pd.DataFrame(rolling_data)
current_time = pd.Timestamp.now().strftime('%d-%m-%Y %H_%M_%S')

#rd_df.to_csv(f"C:/Users/10675048/OneDrive - LTIMindtree/CPG DOMAIN/Sample Data/NPD Data/rolling_months_custom_function_{current_time}.csv")

01-01-2022 <class 'str'>
future: 01-02-2022
future: 01-03-2022
future: 01-04-2022
future: 01-05-2022
01-02-2022 <class 'str'>
future: 01-03-2022
future: 01-04-2022
future: 01-05-2022
future: 01-06-2022
01-03-2022 <class 'str'>
future: 01-04-2022
future: 01-05-2022
future: 01-06-2022
future: 01-07-2022
01-04-2022 <class 'str'>
future: 01-05-2022
future: 01-06-2022
future: 01-07-2022
future: 01-08-2022
01-05-2021 <class 'str'>
future: 01-06-2021
future: 01-07-2021
future: 01-08-2021
future: 01-09-2021
01-05-2022 <class 'str'>
future: 01-06-2022
future: 01-07-2022
future: 01-08-2022
future: 01-09-2022
01-06-2021 <class 'str'>
future: 01-07-2021
future: 01-08-2021
future: 01-09-2021
future: 01-10-2021
01-07-2021 <class 'str'>
future: 01-08-2021
future: 01-09-2021
future: 01-10-2021
future: 01-11-2021
01-08-2021 <class 'str'>
future: 01-09-2021
future: 01-10-2021
future: 01-11-2021
future: 01-12-2021
01-09-2021 <class 'str'>
future: 01-10-2021
future: 01-11-2021
future: 01-12-2021
future: 0

In [11]:
#rd_df.columns

In [12]:
#merged_df = pd.merge(df1, df2, left_on='ID', right_on='CustomerID')
#merged_df = pd.merge(data,rd_df, left_on='variant_combo', right_on='varaintcombo' )
#merged_df.head(10)

#merged_df.to_csv(f"C:/Users/10675048/OneDrive - LTIMindtree/CPG DOMAIN/Sample Data/NPD Data/Merged_rolling_months_custom_function_{current_time}.csv")


In [16]:
df.to_csv(f"C:/Users/10675048/OneDrive - LTIMindtree/CPG DOMAIN/Sample Data/NPD Data/variantGroupedData.csv")

In [19]:
dt = df['Dates'].unique()
print(type(dt))
#rm1 = range(0,4)
#print(rm1)
rm =4
n=4
#range(1,len(dt)-rm)

for i in range(len(dt)-(rm-1)):
    print("i:--",i)
    for d in dt:
        print(d)
        for j in range(n-1):
            print("j:",j)

<class 'numpy.ndarray'>
i:-- 0
01-01-2022
j: 0
j: 1
j: 2
01-02-2022
j: 0
j: 1
j: 2
01-03-2022
j: 0
j: 1
j: 2
01-04-2022
j: 0
j: 1
j: 2
01-05-2021
j: 0
j: 1
j: 2
01-05-2022
j: 0
j: 1
j: 2
01-06-2021
j: 0
j: 1
j: 2
01-07-2021
j: 0
j: 1
j: 2
01-08-2021
j: 0
j: 1
j: 2
01-09-2021
j: 0
j: 1
j: 2
01-10-2021
j: 0
j: 1
j: 2
01-11-2021
j: 0
j: 1
j: 2
01-12-2021
j: 0
j: 1
j: 2
i:-- 1
01-01-2022
j: 0
j: 1
j: 2
01-02-2022
j: 0
j: 1
j: 2
01-03-2022
j: 0
j: 1
j: 2
01-04-2022
j: 0
j: 1
j: 2
01-05-2021
j: 0
j: 1
j: 2
01-05-2022
j: 0
j: 1
j: 2
01-06-2021
j: 0
j: 1
j: 2
01-07-2021
j: 0
j: 1
j: 2
01-08-2021
j: 0
j: 1
j: 2
01-09-2021
j: 0
j: 1
j: 2
01-10-2021
j: 0
j: 1
j: 2
01-11-2021
j: 0
j: 1
j: 2
01-12-2021
j: 0
j: 1
j: 2
i:-- 2
01-01-2022
j: 0
j: 1
j: 2
01-02-2022
j: 0
j: 1
j: 2
01-03-2022
j: 0
j: 1
j: 2
01-04-2022
j: 0
j: 1
j: 2
01-05-2021
j: 0
j: 1
j: 2
01-05-2022
j: 0
j: 1
j: 2
01-06-2021
j: 0
j: 1
j: 2
01-07-2021
j: 0
j: 1
j: 2
01-08-2021
j: 0
j: 1
j: 2
01-09-2021
j: 0
j: 1
j: 2
01-10-2021
j: 0
j: 